# John Pezzuto - HW 7

##### Packages & Data

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from pylab import rcParams
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline


auto = pd.read_csv('Auto.csv', na_values='?')
auto = auto.dropna()
auto['mpg_high'] = auto['mpg'].apply(lambda x: 1 if x >= auto['mpg'].median() else 0)
auto['const'] = 1

In [31]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high,const
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0,1


### a) Fit a logistic model of mpg high

In [72]:
X = auto[['const', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
y = auto['mpg_high'].values

k = 4
kf_log = KFold(n_splits=k, shuffle=True, random_state=15)
kf_log.get_n_splits(X)

MSE_vec_kf = np.zeros(k)
true_MSE_kf = np.zeros(k)
false_MSE_kf = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf_log.split(X):
    print('\nk index =', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    
    true_vals = []
    true_preds = []
    false_vals = []
    false_preds = []
    for i, p in enumerate(y_pred):
        if p == 1:
            true_preds.append(p)
            true_vals.append(y_test[i])
        else:
            false_preds.append(p)
            false_vals.append(y_test[i])
    true_vals = np.asarray(true_vals)
    true_preds = np.asarray(true_preds)
    false_vals = np.asarray(false_vals)
    false_preds = np.asarray(false_preds)
    true_MSE_kf[k_ind] = ((true_vals - true_preds) ** 2).mean()
    false_MSE_kf[k_ind] = ((false_vals - false_preds) ** 2).mean()
    
    print('Test Set MSE:', MSE_vec_kf[k_ind])
    print('mpg_high Error rate(category 1):', true_MSE_kf[k_ind])
    print('mpg_high Error rate(category 0):', false_MSE_kf[k_ind])

    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()

print('\nK-Fold Results:')
print('Test Estimate MSE:', MSE_kf,
      '\nTest estimate MSE Standard Error:', MSE_kf_std)
true_MSE = true_MSE_kf.mean()
false_MSE = false_MSE_kf.mean()
print('mpg_high Error rate(category 1):', true_MSE,
      '\nmpg_high Error rate(category 0):', false_MSE)


k index = 0
Test Set MSE: 0.0918367346939
Error rate(category 1): 0.130434782609
Error rate(category 0): 0.0576923076923

k index = 1
Test Set MSE: 0.102040816327
Error rate(category 1): 0.0816326530612
Error rate(category 0): 0.122448979592

k index = 2
Test Set MSE: 0.132653061224
Error rate(category 1): 0.115384615385
Error rate(category 0): 0.152173913043

k index = 3
Test Set MSE: 0.102040816327
Error rate(category 1): 0.145454545455
Error rate(category 0): 0.046511627907

K-Fold Results:
Test Estimate MSE: 0.107142857143 
Test estimate MSE Standard Error: 0.015306122449
mpg_high Error rate(category 1): 0.118226649127 
mpg_high Error rate(category 0): 0.0947067070586


### b) Fit a random forest model

In [71]:
auto_tree = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)
auto_tree.fit(X, y)
rnd = np.vectorize(lambda x: int(x >= .5))
ypred = rnd(auto_tree.oob_decision_function_[:,1])
MSE = mean_squared_error(y, ypred)
print('MSE=', MSE)
autotreedf = pd.DataFrame({'y':y,'ypred':ypred})
autotreedf['MSE'] = autotreedf.apply(lambda x: (x['y']-x['ypred'])**2, axis=1)
print('mpg_high Error rate(category 1):', autotreedf.groupby(y)['MSE'].mean()[1])
print('mpg_high Error rate(category 0):', autotreedf.groupby(y)['MSE'].mean()[0])

MSE= 0.0790816326531
mpg_high Error rate(category 1): 0.0714285714286
mpg_high Error rate(category 0): 0.0867346938776


### c) Fit a support vector machines

In [74]:
kf_svm = KFold(n_splits=k, shuffle=True, random_state=15)
kf_svm.get_n_splits(X)

MSE_vec_kf = np.zeros(k)
true_MSE_kf = np.zeros(k)
false_MSE_kf = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf_log.split(X):
    print('\nk index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = svm.SVC(kernel='rbf', C=1, gamma=.2)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    
    true_vals = []
    true_preds = []
    false_vals = []
    false_preds = []
    for i, p in enumerate(y_pred):
        if p == 1:
            true_preds.append(p)
            true_vals.append(y_test[i])
        else:
            false_preds.append(p)
            false_vals.append(y_test[i])
    
    if true_vals and true_preds:
        true_vals = np.asarray(true_vals)
        true_preds = np.asarray(true_preds)
        
        true_MSE_kf[k_ind] = ((true_vals - true_preds) ** 2).mean()
    else:
        true_MSE_kf[k_ind] = 0.0
    
    if false_vals and false_preds:
        false_preds = np.asarray(false_preds)
        false_vals = np.asarray(false_vals)
        
        false_MSE_kf[k_ind] = ((false_vals - false_preds) ** 2).mean()
    else:
        false_MSE_kf[k_ind] = 0.0
            
     
    print('Test set MSE:', MSE_vec_kf[k_ind])
    print('mpg_high Error rate(category 1):', true_MSE_kf[k_ind])
    print('mpg_high Error rate(category 0):', false_MSE_kf[k_ind])

    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
true_MSE = true_MSE_kf.mean()
false_MSE = false_MSE_kf.mean()

print('\nK-Fold Results:')
print('Test Estimate MSE:', MSE_kf,
      '\nTest estimate MSE Standard Error:', MSE_kf_std)
true_MSE = true_MSE_kf.mean()
false_MSE = false_MSE_kf.mean()
print('mpg_high Error rate(category 1):', true_MSE,
      '\nmpg_high Error rate(category 0):', false_MSE)


k index= 0
Test set MSE: 0.540816326531
mpg_high Error rate(category 1): 0.552083333333
mpg_high Error rate(category 0): 0.0

k index= 1
Test set MSE: 0.520408163265
mpg_high Error rate(category 1): 0.0
mpg_high Error rate(category 0): 0.520408163265

k index= 2
Test set MSE: 0.520408163265
mpg_high Error rate(category 1): 0.0
mpg_high Error rate(category 0): 0.53125

k index= 3
Test set MSE: 0.448979591837
mpg_high Error rate(category 1): 0.0
mpg_high Error rate(category 0): 0.47311827957

K-Fold Results:
Test Estimate MSE: 0.507653061224 
Test estimate MSE Standard Error: 0.0348846794163
mpg_high Error rate(category 1): 0.138020833333 
mpg_high Error rate(category 0): 0.381194110709


### d) Which is best?

The Random Forest Classifier had the lowest MSE of all 3 models. The Logistic Regression was a close 2nd, and lastly the SVC model was the worst. Furthermore, the Random Forest Classifier lower category 1 and category 0 errors. Again in regards to category 1 and category 0 errors, the Logistic Regression model is close behind and the SVC model remains the worst 